In [1]:
import os

In [2]:
pwd

'c:\\Users\\jruan\\Documents\\End-to-End-Machine-Learning-Project-with-ML-Flow\\research'

In [3]:
os.chdir('../')

In [4]:
pwd

'c:\\Users\\jruan\\Documents\\End-to-End-Machine-Learning-Project-with-ML-Flow'

In [5]:
# def the entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir:Path
    train_data_path:Path
    test_data_path:Path
    model_name:str
    alpha: float # this is the parameters.
    l1_ratio: float # this is the parameters.
    target_column: str # this comes from the schema 

In [6]:
# setting the configuration manager
from mlproject.constants import *
from mlproject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
    
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
    
        create_directories([self.config.artifacts_root])
    

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir= config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path= config.test_data_path,
            model_name= config.model_name,
            alpha= params.alpha,
            l1_ratio = params.l1_ratio,
            target_column=schema.name
        )

        return model_trainer_config

In [8]:
# finding the model components
import pandas as pd
import os 
from mlproject import logger
from sklearn.linear_model import ElasticNet
import joblib

In [12]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column], axis = 1)
        test_x = test_data.drop([self.config.target_column], axis = 1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        lr = ElasticNet(alpha = self.config.alpha, l1_ratio=self.config.l1_ratio, random_state = 42)
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

In [13]:
# preparing the pipeline
try:
    config = ConfigurationManager()
    Model_trainer_config = config.get_model_trainer_config()
    Model_trainer_config = ModelTrainer(config= Model_trainer_config)
    Model_trainer_config.train()

except Exception as e:
    raise e

[2025-01-31 21:31:58,633: INFO: common: yaml file: Config\config.yaml loaded successfully]
[2025-01-31 21:31:58,636: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-31 21:31:58,640: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-31 21:31:58,642: INFO: common: created directory at: artifacts]
[2025-01-31 21:31:58,643: INFO: common: created directory at: artifacts/model_trainer]


In [14]:
# Testing the ConfigurationManager
if __name__ == '__main__':
    try:
        config = ConfigurationManager()
        model_trainer_config = config.get_model_trainer_config()
        print("Model Trainer Config Loaded:", model_trainer_config)
    except Exception as e:
        print("Error:", e)

[2025-01-31 21:32:06,183: INFO: common: yaml file: Config\config.yaml loaded successfully]
[2025-01-31 21:32:06,187: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-31 21:32:06,190: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-31 21:32:06,191: INFO: common: created directory at: artifacts]
[2025-01-31 21:32:06,193: INFO: common: created directory at: artifacts/model_trainer]
Model Trainer Config Loaded: ModelTrainerConfig(root_dir='artifacts/model_trainer', train_data_path='artifacts/data_transformation/train.csv', test_data_path='artifacts/data_transformation/test.csv', model_name='model.joblib', alpha=0.5, l1_ratio=0.7, target_column='quality')
